In [55]:
import requests
import json
import time
import pymongo
from pymongo import MongoClient
import re
import numpy as np
import pandas as pd

a) Started mongodb server locally, and ensured the instance is operational by checking through Studio 3T.


b) We'll now create a connection to our mongodb instance, create a database and insert a document.

In [47]:
def connect_mongo():
    #Create a connection to the local mongodb server instance
    client = MongoClient("localhost", 27017)
    
    #Let us test the collection from our default database 'local'
    status = client['local']
    
    return client, status 

def insert_document (client, name,collection,doc):
    '''
    Let us now insert this document into mongodb, as per documentation, only during this insertion the db will be created
    if not already created. So if it is already created, it will get connect to the database again, and insert the document. But,
    there is a chance it could be duplicated if ran multiple times with the same document (i.e in value sense, not _id).
    '''
    
    db = client[name]
    collection = db[collection]
    doc_insert = collection.insert_one(doc)        

client, status = connect_mongo()

print("Sample status of our connection:", status)

#Document to be inserted 
doc = {"ip": "192.168.1.1", "city": "Davis", "zip": "95616"} 
db_name='msba'; collection = 'ip_addresses'

#Let the create the databse and collection and insert the document 
insert_document(client,db_name,collection,doc) 

#Now print the documents inside this collection 
for doc in client[db_name][collection].find({}):
    print(doc)

Sample status of our connection: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'local')
{'_id': ObjectId('65f180e249d176f6dcd31426'), 'ip': '192.168.1.1', 'city': 'Davis', 'zip': '95616'}


Now we'll do the following,

c) Explored IPStack website, and got the free API access key.

d) First, we'll create the list of ip_addresses that we're interested in tracking their geo-location, this includes my personal ip address as well. 

e) We'll then send requests for each ip_address in the list for their details, and print all the information on the screen. While doing this, we'll also create a json object for each request and store them in an array. 

f) Now, we'll only print the zip and city for these ip addresses. 

g) We'll store these ip_addresses information in our original document format (ip, city, zip) into our collection 

In [42]:
#Create the ip_address list 
ip_add = ["8.8.8.8","128.120.0.25","128.32.12.14","64.165.72.144","135.180.36.253"]

ip_info=[]

def get_info(ip):
    #Send request to ipstack 
    url=f"http://api.ipstack.com/{ip}?access_key=04c6c3ecefa7fb99fbbda7f726677819&output=json"
    time.sleep(5)
    details = requests.get(url)
    return details

for ip in ip_add:
    #This will print the status of the requests
    info = get_info(ip)
    print(ip,info)

    #Let us also parse this response content into a json object 
    doc=json.loads(str(info.text))
    ip_info.append(doc)

8.8.8.8 <Response [200]>
128.120.0.25 <Response [200]>
128.32.12.14 <Response [200]>
64.165.72.144 <Response [200]>
135.180.36.253 <Response [200]>


In [43]:
#Let us just casually check one documents content, just to see how it looks and what fields are fetched by default 

ip_info[0]

{'ip': '8.8.8.8',
 'type': 'ipv4',
 'continent_code': 'NA',
 'continent_name': 'North America',
 'country_code': 'US',
 'country_name': 'United States',
 'region_code': 'OH',
 'region_name': 'Ohio',
 'city': 'Glenmont',
 'zip': '44628',
 'latitude': 40.5369987487793,
 'longitude': -82.12859344482422,
 'location': {'geoname_id': None,
  'capital': 'Washington D.C.',
  'languages': [{'code': 'en', 'name': 'English', 'native': 'English'}],
  'country_flag': 'https://assets.ipstack.com/flags/us.svg',
  'country_flag_emoji': '🇺🇸',
  'country_flag_emoji_unicode': 'U+1F1FA U+1F1F8',
  'calling_code': '1',
  'is_eu': False}}

In [48]:
#Now let us only print ip, zip, and city for all these 5 ips, and we'll also store that as an dic so we can insert that 
#document to our collection based using our function 'insert_document'

for info in ip_info:
    doc = {'ip':info['ip'],'city':info['city'],'zip':info['zip']}
    print(doc)

    #Let us insert this doc into our collection 
    insert_document(client,db_name,collection,doc)

#Let us also verify the documents stored inside our db collection 
print("\nDocs in my MSBA db's ip_addresses collection:\n")
for doc in client[db_name][collection].find({}):
    print(doc)

{'ip': '8.8.8.8', 'city': 'Glenmont', 'zip': '44628'}
{'ip': '128.120.0.25', 'city': 'Davis', 'zip': '95616'}
{'ip': '128.32.12.14', 'city': 'Berkeley', 'zip': '94705'}
{'ip': '64.165.72.144', 'city': 'Florin', 'zip': '95819'}
{'ip': '135.180.36.253', 'city': 'San Francisco', 'zip': '94118'}

Docs in my MSBA db's ip_addresses collection:

{'_id': ObjectId('65f180e249d176f6dcd31426'), 'ip': '192.168.1.1', 'city': 'Davis', 'zip': '95616'}
{'_id': ObjectId('65f180e849d176f6dcd31427'), 'ip': '8.8.8.8', 'city': 'Glenmont', 'zip': '44628'}
{'_id': ObjectId('65f180e849d176f6dcd31428'), 'ip': '128.120.0.25', 'city': 'Davis', 'zip': '95616'}
{'_id': ObjectId('65f180e849d176f6dcd31429'), 'ip': '128.32.12.14', 'city': 'Berkeley', 'zip': '94705'}
{'_id': ObjectId('65f180e849d176f6dcd3142a'), 'ip': '64.165.72.144', 'city': 'Florin', 'zip': '95819'}
{'_id': ObjectId('65f180e849d176f6dcd3142b'), 'ip': '135.180.36.253', 'city': 'San Francisco', 'zip': '94118'}
